In [1]:
from flask import Flask, request, jsonify
import sqlite3

app = Flask(__name__)
DB_NAME = "muhasebe.db"

In [2]:
def veritabani_olustur():
    with sqlite3.connect(DB_NAME) as conn:
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS musteriler (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                ad TEXT,
                email TEXT,
                telefon TEXT
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS odemeler (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                musteri_id INTEGER,
                odeme_tarihi TEXT,
                odeme_tutari REAL,
                aciklama TEXT,
                FOREIGN KEY(musteri_id) REFERENCES musteriler(id)
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS faturalar (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                musteri_id INTEGER,
                fatura_tarihi TEXT,
                toplam_tutar REAL,
                aciklama TEXT,
                FOREIGN KEY(musteri_id) REFERENCES musteriler(id)
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS cari_hareket (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                musteri_id INTEGER,
                tarih TEXT,
                islem_turu TEXT CHECK(islem_turu IN ('borc', 'alacak')),
                tutar REAL,
                aciklama TEXT,
                FOREIGN KEY(musteri_id) REFERENCES musteriler(id)
            )
        ''')

## Müşteri Endpointleri 

In [3]:
@app.route('/musteri_ekle', methods=['POST'])
def musteri_ekle():
    data = request.get_json()
    if not data or not all(k in data for k in ('ad', 'email', 'telefon')):
        return jsonify({'hata': 'Eksik alan'}), 400
    with sqlite3.connect(DB_NAME) as conn:
        conn.execute('INSERT INTO musteriler (ad, email, telefon) VALUES (?, ?, ?)',
                     (data['ad'], data['email'], data['telefon']))
        conn.commit()
    return {'mesaj': 'Müşteri eklendi'}

In [4]:
@app.route('/musteri_guncelle/<int:musteri_id>', methods=['PUT'])
def musteri_guncelle(musteri_id):
    data = request.get_json()
    if not data or not all(k in data for k in ('ad', 'email', 'telefon')):
        return jsonify({'hata': 'Eksik alan'}), 400
    with sqlite3.connect(DB_NAME) as conn:
        conn.execute('''
            UPDATE musteriler
            SET ad = ?, email = ?, telefon = ?
            WHERE id = ?
        ''', (data['ad'], data['email'], data['telefon'], musteri_id))
        conn.commit()
    return {'mesaj': 'Müşteri bilgileri güncellendi'}

In [5]:
@app.route('/musteriler', methods=['GET'])
def musteriler():
    with sqlite3.connect(DB_NAME) as conn:
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM musteriler')
        musteriler = [dict(row) for row in cursor.fetchall()]
    return jsonify(musteriler)

In [6]:
@app.route('/musteri/<int:musteri_id>', methods=['GET'])
def musteri_bilgisi(musteri_id):
    with sqlite3.connect(DB_NAME) as conn:
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM musteriler WHERE id = ?', (musteri_id,))
        row = cursor.fetchone()
        if not row:
            return jsonify({'hata': 'Müşteri bulunamadı'}), 404
    return jsonify(dict(row))

## Fatura Endpoitleri

In [7]:
@app.route('/fatura_ekle', methods=['POST'])
def fatura_ekle():
    data = request.get_json()
    if not data or not all(k in data for k in ('musteri_id', 'fatura_tarihi', 'toplam_tutar', 'aciklama')):
        return jsonify({'hata': 'Eksik alan'}), 400
    with sqlite3.connect(DB_NAME) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO faturalar (musteri_id, fatura_tarihi, toplam_tutar, aciklama) VALUES (?, ?, ?, ?)',
                       (data['musteri_id'], data['fatura_tarihi'], data['toplam_tutar'], data['aciklama']))
        # Cari hareket olarak borç ekle
        cursor.execute('INSERT INTO cari_hareket (musteri_id, tarih, islem_turu, tutar, aciklama) VALUES (?, ?, ?, ?, ?)',
               (data['musteri_id'], data['fatura_tarihi'], 'borc', data['toplam_tutar'], f"Fatura ID {cursor.lastrowid}: {data['aciklama']}"))

        conn.commit()
    return {'mesaj': 'Fatura ve cari hareket eklendi'}

In [8]:
@app.route('/fatura_guncelle/<int:fatura_id>', methods=['PUT'])
def fatura_guncelle(fatura_id):
    data = request.get_json()
    if not data or not all(k in data for k in ('musteri_id', 'fatura_tarihi', 'toplam_tutar', 'aciklama')):
        return jsonify({'hata': 'Eksik alan'}), 400
    with sqlite3.connect(DB_NAME) as conn:
        cursor = conn.cursor()
        # Önce faturayı güncelle
        cursor.execute('''
            UPDATE faturalar
            SET musteri_id = ?, fatura_tarihi = ?, toplam_tutar = ?, aciklama = ?
            WHERE id = ?
        ''', (data['musteri_id'], data['fatura_tarihi'], data['toplam_tutar'], data['aciklama'], fatura_id))

        # Cari hareket güncelle (Fatura ID'ye göre arama)
        cursor.execute('''
            UPDATE cari_hareket
            SET musteri_id = ?, tarih = ?, tutar = ?, aciklama = ?
            WHERE islem_turu = 'borc' AND aciklama LIKE ?
        ''', (data['musteri_id'], data['fatura_tarihi'], data['toplam_tutar'], f"Fatura ID {fatura_id}: {data['aciklama']}", f"%Fatura ID {fatura_id}:%"))
        # Not: Eğer cari_harekette fatura_id ile bağlantı yoksa, bu kısım daha gelişmiş hale getirilmeli
        conn.commit()
    return {'mesaj': 'Fatura ve cari hareket güncellendi'}

In [9]:
@app.route('/musteri/<int:musteri_id>/faturalar', methods=['GET'])
def musterinin_faturalari(musteri_id):
    with sqlite3.connect(DB_NAME) as conn:
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM faturalar WHERE musteri_id = ?', (musteri_id,))
        faturalar = [dict(row) for row in cursor.fetchall()]
    return jsonify(faturalar)


## Ödeme Endpointleri

In [10]:
@app.route('/odeme_ekle', methods=['POST'])
def odeme_ekle():
    data = request.get_json()
    if not data or not all(k in data for k in ('musteri_id', 'odeme_tarihi', 'odeme_tutari', 'aciklama')):
        return jsonify({'hata': 'Eksik alan'}), 400
    with sqlite3.connect(DB_NAME) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO odemeler (musteri_id, odeme_tarihi, odeme_tutari, aciklama) VALUES (?, ?, ?, ?)',
                       (data['musteri_id'], data['odeme_tarihi'], data['odeme_tutari'], data['aciklama']))
        # Cari hareket olarak alacak ekle
        cursor.execute('INSERT INTO cari_hareket (musteri_id, tarih, islem_turu, tutar, aciklama) VALUES (?, ?, ?, ?, ?)',
               (data['musteri_id'], data['odeme_tarihi'], 'alacak', data['odeme_tutari'], f"Ödeme ID {cursor.lastrowid}: {data['aciklama']}"))

        conn.commit()
    return {'mesaj': 'Ödeme ve cari hareket eklendi'}

In [11]:
@app.route('/odeme_guncelle/<int:odeme_id>', methods=['PUT'])
def odeme_guncelle(odeme_id):
    data = request.get_json()
    if not data or not all(k in data for k in ('musteri_id', 'odeme_tarihi', 'odeme_tutari', 'aciklama')):
        return jsonify({'hata': 'Eksik alan'}), 400
    with sqlite3.connect(DB_NAME) as conn:
        cursor = conn.cursor()
        # Ödeme güncelle
        cursor.execute('''
            UPDATE odemeler
            SET musteri_id = ?, odeme_tarihi = ?, odeme_tutari = ?, aciklama = ?
            WHERE id = ?
        ''', (data['musteri_id'], data['odeme_tarihi'], data['odeme_tutari'], data['aciklama'], odeme_id))

        cursor.execute('''
            UPDATE cari_hareket
            SET musteri_id = ?, tarih = ?, tutar = ?, aciklama = ?
            WHERE islem_turu = 'alacak' AND aciklama LIKE ?
        ''', (data['musteri_id'], data['odeme_tarihi'], data['odeme_tutari'], f"Ödeme ID {odeme_id}: {data['aciklama']}", f"%Ödeme ID {odeme_id}:%"))

        # Not: Daha gelişmiş ilişkilendirme yapılabilir
        conn.commit()
    return {'mesaj': 'Ödeme ve cari hareket güncellendi'}

In [12]:
@app.route('/musteri/<int:musteri_id>/odemeler', methods=['GET'])
def musterinin_odemeleri(musteri_id):
    with sqlite3.connect(DB_NAME) as conn:
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM odemeler WHERE musteri_id = ?', (musteri_id,))
        odemeler = [dict(row) for row in cursor.fetchall()]
    return jsonify(odemeler)

## Cari Hesap Endpointi

In [13]:
@app.route('/cari_bakiye/<int:musteri_id>', methods=['GET'])
def cari_bakiye(musteri_id):
    with sqlite3.connect(DB_NAME) as conn:
        cursor = conn.cursor()
        cursor.execute('''
            SELECT 
                SUM(CASE WHEN islem_turu = 'borc' THEN tutar ELSE 0 END) AS toplam_borc,
                SUM(CASE WHEN islem_turu = 'alacak' THEN tutar ELSE 0 END) AS toplam_alacak
            FROM cari_hareket
            WHERE musteri_id = ?
        ''', (musteri_id,))
        row = cursor.fetchone()
        toplam_borc = row[0] if row[0] else 0
        toplam_alacak = row[1] if row[1] else 0
        bakiye = toplam_borc - toplam_alacak
    return jsonify({'toplam_borc': toplam_borc, 'toplam_alacak': toplam_alacak, 'bakiye': bakiye})

In [ ]:
if __name__ == '__main__':
    veritabani_olustur()
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [16/Jun/2025 05:07:21] "GET /musteriler HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2025 05:07:29] "GET /musteri/1/faturalar HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2025 05:07:36] "PUT /fatura_guncelle/1 HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2025 05:07:39] "PUT /odeme_guncelle/1 HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2025 05:07:41] "GET /musteri/1/odemeler HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2025 05:07:46] "GET /cari_bakiye/1 HTTP/1.1" 200 -
